**Load Libs**

In [23]:
import os
import requests
import pandas as pd
import time
import json

**Paths**

In [30]:
TOKEN = 'data\\token'
TO_SCRAP = 'data\\to_scrap'
SCRAPED = 'data\\scraped'
ERRORS = 'data\\errors'

In [34]:
print(os.path.abspath(TOKEN))

C:\Users\juanm\OneDrive\Bureau\ESGI - Projets\4IABD\Projet Annuel\data\token


**Generate Token For Spotify API Requests**

In [ ]:
# Generate token
def generate_token() -> str:
    f = open(f'{TOKEN}\\auth_spotify_api.json')
    auth_spotify_api = json.load(f)
    response = requests.post('https://accounts.spotify.com/api/token',
                             data={
                                 'Content_type': 'application/x-www-form-urlencoded',
                                 'grant_type': 'client_credentials',
                                 'client_id': auth_spotify_api.get('client_id'),
                                 'client_secret': auth_spotify_api.get('client_secret')
                             }).json()['access_token']
    return response

In [ ]:
def regen_token(token: str, timestamp: float) -> tuple[str, float]:
    t = time.time()
    if t - timestamp >= 3600:
        return generate_token(), t
    return token, timestamp

In [ ]:
spotify_api = 'https://api.spotify.com/v1'

**Load Dataset To Scrap**

In [ ]:
df = pd.read_csv(f'{TO_SCRAP}\\custom_music_dataset.csv')

**Retrieve Dates And SongID**

In [ ]:
# Create empty csv with columns
pd.DataFrame({
    'artist_name': pd.Series(dtype='str'),
    'track_name': pd.Series(dtype='str'),
    'release_date': pd.Series(dtype='str'),
    'genre': pd.Series(dtype='str'),
    'spotify_id': pd.Series(dtype='str')
}).to_csv(f'{SCRAPED}\\complete_custom_dataset.csv', index=False)

In [9]:
timestamp = time.time()
token = generate_token()
for rows in df.values:
    token, timestamp = regen_token(token, timestamp)
    response = requests.get(f'{spotify_api}/search?q=track:{rows[0]}%20artist:{rows[1]}&type=track',
                            headers={
                                'Authorization': f'Bearer {token}'
                            })
    if response.status_code == 429:
        print('Too Many Request')
        break
    if response.status_code == 200 and response.json()['tracks']['total'] > 0:
        album_dict = response.json()['tracks']['items'][0]['album']
        pd.DataFrame({'artist_name': rows[1],
                      'track_name': rows[0],
                      'release_date': f"{album_dict['release_date'][:3]}0" if 'genres' in album_dict
                         else '',
                      'genre': album_dict['genres'] if 'genres' in album_dict
                         else '',
                      'spotify_id': response.json()['tracks']['items'][0]['id']}, index=[0]) \
            .to_csv(f'{SCRAPED}\\complete_custom_dataset.csv', index=False, header=False, mode='a')
    else:
        pd.DataFrame({'artist_name': rows[1],
                      'track_name': rows[0],
                      'release_date': '',
                      'genre': '',
                      'error_type': response.status_code}, index=[0]) \
            .to_csv(f'{ERRORS}\\error_scrapping_custom.csv', index=False, header=False, mode='a')

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3APoor+Little+Fool+artist%3ARicky+Nelson&type=track&offset=0&limit=20', 'items': [{'album': {'album_group': 'album', 'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/73sSFVlM6pkweLXE8qw1OS'}, 'href': 'https://api.spotify.com/v1/artists/73sSFVlM6pkweLXE8qw1OS', 'id': '73sSFVlM6pkweLXE8qw1OS', 'name': 'Ricky Nelson', 'type': 'artist', 'uri': 'spotify:artist:73sSFVlM6pkweLXE8qw1OS'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'ET', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM

KeyError: 'genres'